In [25]:
# Extract samples generated from Milestone3_Notebook.ipynb.
!pip install pyspark
!tar -xf sample0.tar.gz
!tar -xf sample1.tar.gz
!tar -xf sample2.tar.gz
!tar -xf sample3.tar.gz
!tar -xf sample4.tar.gz
!tar -xf sample5.tar.gz
!tar -xf sample6.tar.gz

In [26]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
try:
  sc.stop()
except:
  pass
sc = SparkContext(master = 'local[3]')
ss = SparkSession(sc)

In [31]:
# Create a dataframe for each sample and show the first one.
dfs = []
dfs.append(ss.createDataFrame(sc.pickleFile('./sample0')))
dfs[0].show(dfs[0].count())

+-----------+--------------------+-----+----------+--------------------+--------------------+--------------------+------+-------------+--------------------+--------------------+----------+-----------+--------+------------+--------------------+-------------------+-------------------+------+-------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+-----------------+------------+------------------+-------------------+-----------------------+-------------------+--------------+-------------------+---------------+----------------------+-----------------------+---------------------------+--------------------+-------------------+------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+----------------+---------------+--------------+----------------+--------------------+----------+
|         ID|               title

In [35]:
# Append dataframes of the other samples.
sample_schema = dfs[0].schema
for i in range(1, 7):
  dfs.append(ss.createDataFrame(sc.pickleFile('./sample' + str(i)),
                                schema = sample_schema))

In [36]:
dfs[1].show(dfs[1].count())

+-----------+--------------------+-----+----------+--------------------+--------------------+--------------------+------------------+-------------+--------------------+--------------------+----------+-----------+--------+------------+-----------------+---------------+-------------------+-------------------+-----------+-------------------+--------------+--------------------+-------------------+--------------------+-------------------+-------------------+------------------+-----------------+------------------+-------------------+-----------------------+-------------------+-------------------+-------------------+-------------------+----------------------+-----------------------+---------------------------+--------------------+-------------------+-------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+-----------------+----------------+--------------+----------------+--------------------+----------+

In [37]:
dfs[2].show(dfs[2].count()) # One sample is NOT a cluster, this one can and
                            # should be ignored.

+---------+--------------------+-----+----------+--------------------+-------+------------+-------------+-----------+-----------+-----------+----------+-----------+--------+------------+-----------------+---------------+---------+------+-----------+-------+--------------+---------------+-------------------+-----------+----------+--------+-----------------+------------+------------------+-------------------+-----------------------+------------------+----------------+-------------------+------------------+----------------------+-----------------------+---------------------------+-------------------+------------------+-----------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+-----------------+----------------+---------------+----------------+--------------------+----------+
|       ID|               title| rank|      date|              artist|    url|      region|        chart|      trend|   track_id| 

In [38]:
dfs[3].show(dfs[3].count())

+-----------+--------------------+----+--------------+------+----------+--------------------+--------------------+-----+-------------+-------+----------+-----------+--------+--------------+-----------------+---------------+---------+------+-----------+------------------+------------------+---------------+-------------------+-----------+------------------+--------------------+--------------------+-----------------+------------------+-------------------+-----------------------+------------------+----------------+-------------------+------------------+----------------------+-----------------------+---------------------------+-------------------+------------------+--------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+----------------+----------------+--------------+----------------+--------------------+----------+
|         ID|               title|rank|          date|artist|       url|           

In [39]:
dfs[4].show(dfs[4].count())

+-----------+-------------------------------+-----+----------+--------------------+--------------------+--------------+-------+-------------+--------------------+------------------------+----------+-----------+--------+------------+--------------------+-------------------+-------------------+------+-------------------+-------+--------------------+-------------------+--------------------+-------------------+-------------------+------------------+-----------------+------------+------------------+-------------------+-----------------------+-------------------+--------------+-------------------+---------------+----------------------+-----------------------+---------------------------+-------------------+-------------------+------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+----------------+---------------+--------------+----------------+--------------------+----------+
|         ID|             

In [40]:
dfs[5].show(dfs[5].count())

+-----------+--------------------+----+----------+--------------------+--------------------+--------------------+-------+-------------+--------------------+-------------------------+----------+-----------+--------+------------+--------------------+-------------------+-------------------+------+-------------------+-------+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+-----------------+------------+------------------+-------------------+-----------------------+-------------------+----------------+-------------------+------------------+----------------------+-----------------------+---------------------------+-------------------+-------------------+------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+----------------+---------------+--------------+----------------+--------------------+----------+
|         ID|            

In [41]:
dfs[6].show(dfs[6].count())

+-----------+--------------------+-----+----------+--------------------+--------------------+--------------------+-------+-------------+--------------------+--------------------+----------+-----------+--------+------------+--------------------+-------------------+--------------------+------+-------------------+-------+--------------------+--------------------+-------------------+--------------------+-------------------+------------------+-----------------+------------+------------------+-------------------+-----------------------+--------------------+--------------+-------------------+---------------+----------------------+-----------------------+---------------------------+--------------------+-------------------+------------------+-------------------------+--------------------+--------------------+--------------+-------------+-------------+----------------+----------------+---------------+--------------+----------------+--------------------+----------+
|         ID|               tit

In [63]:
# A labeling mechanism designed to label our unsupervised dataset in a manner
# best aligned with the clusters generated from Milestone3_Notebook.ipynb. This
# is a preliminary mechanism based on high-level human judgment--as such, it is
# imprecise and is likely to require far more rigorous tuning in our future
# analyses.
def assign_genre(row):
    if row['af_danceability'] > 0.65 and row['af_energy'] > 0.45:
        return 'Dance'
    elif row['af_tempo'] < 110 and row['af_energy'] < 0.55:
        return 'Ballad'
    elif row['af_energy'] > 0.65 and row['af_danceability'] < 0.45:
        return 'Rock'
    elif row['af_liveness'] > 0.75:
        return 'Jazz/Blues'
    elif row['af_speechiness'] > 0.25 and row['af_danceability'] > 0.55:
        return 'Hip-Hop'
    elif row['af_acousticness'] > 0.45 and row['af_energy'] < 0.45:
        return 'Acoustic/Folk'
    else:
        return 'Other'  # Fallback category for songs that do not fit the above categories

In [64]:
# Apply the function to your dataframes
pdfs = []
for df in dfs:
  df2p = df.toPandas()
  df2p['genre'] = df2p.apply(assign_genre, axis=1)
  pdfs.append(df2p)

In [65]:
# Get the genre counts for each dataframe.
genre_stats = dict()
for i in range(len(pdfs)):
  if not i == 2:
    genre_stats[i] = pdfs[i][['ID', 'genre']].groupby('genre').count()

In [66]:
# Print results.
for i in genre_stats:
  print(f'Results for sample {i}:\n{genre_stats[i]}\n\n')

Results for sample 0:
        ID
genre     
Ballad   1
Dance   32
Other   11
Rock     2


Results for sample 1:
        ID
genre     
Ballad   6
Other   45


Results for sample 3:
            ID
genre         
Jazz/Blues  45


Results for sample 4:
               ID
genre            
Acoustic/Folk   9
Ballad         14
Dance           5
Hip-Hop         1
Other           6


Results for sample 5:
       ID
genre    
Dance  33
Other  14


Results for sample 6:
               ID
genre            
Acoustic/Folk  10
Ballad          4
Dance          11
Other          17
Rock            5




# Cluster Sampling Results:
This first evaluation of our clusters is based on a simple, preliminary, arbitrarily chosen labeling function that yields uninformative results. More specifically, the labeling function does not define genres such that the clusters correspond to specific genres. A primary goal of our future analyses will therefore include building a labeling function that can define an abstract label variable (possibly, but not necessarily corresponding to genre) that the clusters may better describe. To do so, we propose to generalize upon our first draft of the labeling function, perhaps incorporating other features and applying hyperparameter tuning to optimize its definition. In this way, we hope that the labeling function will be able to elucidate the kind of trends our clustering model captures and whether these trends are desirable for our purposes.